In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [64]:
import torch
import torch.nn as nn
import numpy as np

#### Generate some data

byte-arrays - N_BYTES bytes

In [24]:
N_BYTES = 10
N_BITS = 8*N_BYTES 

N_SAMPLES = 100

In [25]:
X = np.random.randint(low=0, high=2, size=(N_SAMPLES, N_BITS))

In [37]:
y = X.sum(axis=1)

In [40]:
y

array([34, 43, 32, 43, 39, 43, 43, 37, 41, 38, 39, 41, 37, 45, 41, 42, 41,
       33, 36, 37, 38, 43, 39, 49, 44, 47, 43, 43, 35, 45, 49, 30, 38, 43,
       48, 34, 37, 37, 49, 42, 42, 37, 43, 39, 44, 44, 34, 35, 45, 34, 39,
       43, 47, 39, 42, 33, 45, 41, 33, 36, 36, 43, 44, 44, 38, 45, 48, 45,
       39, 46, 38, 39, 46, 43, 40, 32, 40, 33, 36, 40, 38, 34, 38, 35, 40,
       41, 38, 32, 38, 42, 40, 35, 37, 43, 41, 45, 40, 34, 41, 42])

In [33]:
ARG1_LOC = 0 #byte loc
ARG2_LOC = 1 #byte loc
#VAL_LOC = 4 #byte loc

assert ARG1_LOC < N_BYTES and ARG2_LOC < N_BYTES and VAL_LOC < N_BYTES



In [34]:
#TODO - do the addition

#use np

### Self-attention

1. convert bit vectors to byte vectors
2. map each unique byte value -> embedding vector (untrained)
3. computing keys, values and queries

In [44]:
r = X[0]
r

array([1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1,
       1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1,
       1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0])

#### Step 1

In [88]:
#use numpy for this
print(r)
r_byte = []
for i in range(0, len(r), 8):
    r_byte.append(int("".join([str(x) for x in r[i:i+8]]), base=2))

r_byte = torch.from_numpy(np.array(r_byte))
r_byte

[1 0 1 0 0 1 1 0 0 0 1 1 0 1 0 0 1 0 0 1 0 1 1 1 0 0 0 1 1 1 0 1 1 0 0 1 0
 0 1 0 0 1 0 1 1 1 1 0 1 0 0 0 0 0 1 0 0 0 0 1 1 0 1 0 1 1 0 0 0 0 1 0 1 0
 0 0 1 0 0 0]


tensor([166,  52, 151,  29, 146,  94, 130,  26, 194, 136])

#### Step 2

In [102]:
EMB_DIM = 16

In [103]:
emb = nn.Embedding(num_embeddings=256, embedding_dim=EMB_DIM)
pos_enc = nn.Embedding(num_embeddings=N_BYTES, embedding_dim=EMB_DIM)

exercise: plot histogram of values across all embeddings (256*16 numbers)

In [90]:
r_byte = r_byte.unsqueeze(0)

In [91]:
r_byte.shape

torch.Size([1, 10])

In [92]:
emb(r_byte).shape

torch.Size([1, 10, 16])

In [100]:
# positional encoding
#emb(r_byte) + pos_enc(torch.arange(10)).unsqueeze(0)

Step 3

goal:

- each byte B looks at every other byte C_i, assigns a weight to it w_i, and compute the weighted average of values
- sum(w_i v_i)

In [112]:
#compute the value of each byte
VAL_DIM = 32

l_v = nn.Linear(in_features=EMB_DIM, out_features=VAL_DIM) #values
l_k = nn.Linear(in_features=EMB_DIM, out_features=VAL_DIM) #keys
l_q = nn.Linear(in_features=EMB_DIM, out_features=VAL_DIM) #queries

In [110]:
r_byte.shape

torch.Size([1, 10])

In [108]:
emb(r_byte).shape

torch.Size([1, 10, 16])

In [113]:
l_v(emb(r_byte)).shape

torch.Size([1, 10, 32])

In [115]:
v = l_v(emb(r_byte)) #value
k = l_k(emb(r_byte)) #key
q = l_q(emb(r_byte)) #query